In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
import os
for dirname, _, filename in os.walk("/content/drive/MyDrive/input"):
  for files in filename:
    print(os.path.join(dirname, files))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
%matplotlib inline
import cv2
from tqdm import tqdm_notebook as tqdm

# Data Reading

In [ ]:
train_data = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/input/train.csv"))
test_data = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/input/test.csv"))

In [ ]:
print("Training data shape : = {}".format(train_data.shape))
print("Test data shape : = {}".format(test_data.shape))

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
image_folder_path = "/content/drive/MyDrive/input/images"

In [ ]:
arr = train_data["image_id"]
train_images = [i for i in arr]

arr = test_data["image_id"]
test_images = [i for i in arr]

In [ ]:
def load_image(image_id) :
  IMAGE_SIZE = (224, 224, 3)
  image_path = image_folder_path+'/' +image_id +".jpg"
  image = cv2.imread(image_path)
  image = resize(image, IMAGE_SIZE)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  return image

def resize(image, image_size):
  image = cv2.resize(image, (image_size[0], image_size[1]), interpolation = cv2.INTER_AREA)
  return image

# Exploratory Data Analysis

When we’re getting started with a machine learning (ML) project, one critical principle to keep in mind is that data is everything. It is often said that if ML is the rocket engine, then the fuel is the (high-quality) data fed to ML algorithms. However, deriving truth and insight from a pile of data can be a complicated and error-prone job. To have a solid start for our ML project, it always helps to analyze the data up front.

During EDA, it’s important that we get a deep understanding of:

* The properties of the data, such as schema and statistical properties;
* The quality of the data, like missing values and inconsistent data types;
* The predictive power of the data, such as correlation of features against target.

In [ ]:
def extract_classes(s):
  """
  s can be either of the four classes mentioned above.
  """
  t = train_data[train_data[s] == 1]
  arr = t["image_id"]
  images = [i for i in tqdm(arr)]
  train_images = [load_image(i) for i in tqdm(images)]
  return train_images

classes = ["healthy", "multiple_diseases", "rust", "scab"]

In [ ]:
count_healthy = len(train_data[train_data["healthy"] == 1])
count_diseased = len(train_data[train_data["multiple_diseases"] == 1])
count_rust = len(train_data[train_data["rust"] == 1])
count_scab = len(train_data[train_data["scab"] == 1])

print(count_healthy)
print(count_diseased)
print(count_rust)
print(count_scab)
print(count_healthy + count_diseased + count_rust +  count_scab)

In [ ]:
# observe number of cases present in each class
labels = ["Healthy", "Multiple Diseased", "Rust", "Scab"]
counts = [count_healthy, count_diseased, count_rust, count_scab]
explode = (0.05, 0.05, 0.05, 0.05)
fig, ax = plt.subplots(figsize = (20, 12))
ax.pie(counts, explode = explode, labels = labels, shadow = True, startangle = 90)
ax.axis("equal") # equal aspect ratio ensures pie graph is drawn as circle

Clearly, there is a class imbalance problem which needs to be addressed during the deep learning model construction.

In [ ]:
red , green, blue = [], [], []

# Analysing the color channel distribution in each Image

In [ ]:
def load_image(image_id):
    try:
        image_path = image_folder_path+ "/"+ image_id + ".jpg"
        print(image_path)
        IMAGE_SIZE = (224, 224, 3)

        image = cv2.imread(image_path)
        image = resize(image, IMAGE_SIZE)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image
    except Exception as e:
        print(f"Error loading image {image_id}: {e}")
        return None

In [ ]:
def extract_classes(s):
  """
  s can be either of the four classes mentioned above.
  """
  t = train_data[train_data[s] == 1]
  arr = t["image_id"]
  images = [i for i in tqdm(arr)]
  print(images)
  train_images = []
  for i in images:
    x = load_image(i)
    train_images.append(x)


  train_images = [load_image(i) for i in tqdm(images)]
  return train_images

classes = ["healthy", "multiple_diseases", "rust", "scab"]

In [ ]:
healthy = extract_classes("healthy")
for image in healthy :
    mean_red = np.mean(image[:,:,0])
    mean_green = np.mean(image[:,:,1])
    mean_blue = np.mean(image[:,:,2])

    red.append(mean_red)
    green.append(mean_green)
    blue.append(mean_blue)

healthy_image_1 = healthy[100]
healthy_image_2 = healthy[200]
healthy_image_3 = healthy[300]
del healthy # free memory

md = extract_classes("multiple_diseases")
for image in md :
    mean_red = np.mean(image[:,:,0])
    mean_green = np.mean(image[:,:,1])
    mean_blue = np.mean(image[:,:,2])

    red.append(mean_red)
    green.append(mean_green)
    blue.append(mean_blue)
md_image_1 = md[1]
md_image_2 = md[5]
md_image_3 = md[10]
del md # free memory

rust = extract_classes("rust")
for image in rust :
    mean_red = np.mean(image[:,:,0])
    mean_green = np.mean(image[:,:,1])
    mean_blue = np.mean(image[:,:,2])

    red.append(mean_red)
    green.append(mean_green)
    blue.append(mean_blue)
rust_image_1 = rust[10]
rust_image_2 = rust[20]
rust_image_3 = rust[30]
del rust # free memory

scab = extract_classes("healthy")
for image in scab :
    mean_red = np.mean(image[:,:,0])
    mean_green = np.mean(image[:,:,1])
    mean_blue = np.mean(image[:,:,2])

    red.append(mean_red)
    green.append(mean_green)
    blue.append(mean_blue)
scab_image_1 = scab[10]
scab_image_2 = scab[20]
scab_image_3 = scab[30]
del scab # free memory

image_collection = [healthy_image_1, healthy_image_2, healthy_image_3,
                   md_image_1, md_image_2, md_image_3,
                   rust_image_1, rust_image_2, rust_image_3,
                   scab_image_1, scab_image_2, scab_image_3]

In [ ]:
fig, ax = plt.subplots(nrows = 4, ncols = 3, figsize = (25, 15))
for i in range(12):
    ax[i//3, i%3].imshow(image_collection[i])

**Inference** :
* Rust leaves has brownish- yellowish patches
* Scab leaves have brown stains.

# Skewness in EDA :

Skewness is the measure of symmetry or asymmetry of a data distribution. A distribution or data set is said to be symmetric if it looks same to the left and right point of the center.

Types of Skewness :

Skewness is generally classified into 2 broad categories-

* Right skewness or Positive skewness
* Left skewness or Negative skewness

![image.png](attachment:image.png)

It is very difficult to interpret and analyse the data which is skewed.

# Kurtosis :

Kurtosis is the characteristics of being flat or peaked. It is a measure whether data is heavy- tailed or light-tailed in a normal distribution

A large kurtosis value often mean that the tails of the distributions are getting toward more extreme values than the tails of normal distributions. This may lead to a length of 6 or 7 standard deviation from the mean. Similarly, If the kurtosis value is very low, then the tails of the distributions will be less lengthier than the those of a normal distribution (less than 3 standard deviation).

![image.png](attachment:image.png)

A large value of kurtosis is often considered as more risky because data may tend to give an outlier value as outcome with greater distance from the mean if applied to any machine learning algorithm.

In [ ]:
# red channel plot
range_of_spread = max(red) - min(red)
plt.figure(figsize = (12, 8))
plt.rc('font', weight='bold')
sns.set_style("whitegrid")
fig = sns.distplot(red,  hist = True, kde = True, label = "Red Channel intensities", color = "r")
fig.set(xlabel = "Mean red channel intensities observed in each image (Sample size = 1000)", ylabel = "Probability Density")
plt.legend()
print("The range of spread = {:.2f}".format(range_of_spread))

In [ ]:
# Green channel plot
range_of_spread = max(green) - min(green)
plt.figure(figsize = (12, 8))
plt.rc('font', weight='bold')
sns.set_style("whitegrid")
fig = sns.distplot(green,  hist = True, kde = True, label = "Green Channel intensities", color = "g")
fig.set(xlabel = "Mean green channel intensities observed in each image (Sample size = 1000)", ylabel = "Probability Density")
plt.legend()
print("The range of spread = {:.2f}".format(range_of_spread))

In [ ]:
# Blue channel plot
range_of_spread = max(blue) - min(blue)
plt.figure(figsize = (12, 8))
plt.rc('font', weight='bold')
sns.set_style("whitegrid")
fig = sns.distplot(blue,  hist = True, kde = True, rug = False, label = "Blue Channel intensities", color = "b")
fig.set(xlabel = "Mean blue channel intensities observed in each image (Sample size = 1000)", ylabel = "Probability Density")
plt.legend()
print("The range of spread = {:.2f}".format(range_of_spread))

**Inference** :

* Red channel has positive skew, meaning the values are more concentrated at intensities lower than mean(somewhere around 90).
* Green channel is negative skew, meaning the values are more concentrated at intentities higher than mean(somewhere in the range 130-150). This also means that green channel is more pronounced than red in the sample image set; and thereby the whole data set as they come from the same distribution. This makes sense as images are that of leaves!
* Similarily, blue channel has a slight positive skew and is very well distributed.
* The distribution of red and green color channels appears to be mesokurtic, aka normally distributed having k = 0 whereas the blue one appears to be relatively platykurtic having k < 0. Therefore out of the three colors, blue channel appears to be the most different one(relative outlier in the RGB color space).

In [ ]:
sample_image = rust_image_1
plt.figure(figsize = (12, 8))
plt.imshow(sample_image)

# Image Denoising :

Many image smoothing techniques like Gaussian Blurring, Median Blurring etc were good to some extent in removing small quantities of noise. In those techniques, we took a small neighbourhood around a pixel and performed some operations like gaussian weighted average, median of the values etc to replace the central element. In short, noise removal at a pixel was local to its neighbourhood.

There is a property of noise. Noise is generally considered to be a random variable with zero mean.

Suppose we hold a static camera to a certain location for a couple of seconds. This will give us plenty of frames, or a lot of images of the same scene. Then averaging all the frames, we compare the final result and first frame. Reduction in noise would be easily observed.

So idea is simple, we need a set of similar images to average out the noise. Considering a small window (say 5x5 window) in the image, chance is large that the same patch may be somewhere else in the image. Sometimes in a small neighbourhood around it. Hence, using these similar patches together averaging them can lead to an efficient denoised image.

This method is Non-Local Means Denoising. It takes more time compared to blurring techniques, but the result are very satisfying.

Denoising illustration :

![image.png](attachment:image.png)

## OpenCV implementation of the aforementioned approach :

cv2.fastNlMeansDenoisingColored() - Works on Colored images cv2.fastNlMeansDenoising() - Works on graysacle images

Common arguments are:

h : parameter deciding filter strength. Higher h value removes noise better, but removes details of image also. (10 is ok)
hForColorComponents : same as h, but for color images only. (normally same as h)
templateWindowSize : should be odd. (recommended 7)
searchWindowSize : should be odd. (recommended 21)

In [ ]:
def non_local_means_denoising(image) :
    denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    return denoised_image

In [ ]:
denoised_image = non_local_means_denoising(sample_image)

plt.figure(figsize = (12, 8))
plt.subplot(1,2,1)
plt.imshow(sample_image, cmap = "gray")
plt.grid(False)
plt.title("Normal Image")

plt.subplot(1,2,2)
plt.imshow(denoised_image, cmap = "gray")
plt.grid(False)
plt.title("Denoised image")
# Automatically adjust subplot parameters to give specified padding.
plt.tight_layout()

# Edge detection Using Sobel filter :
Edge detection is one of the fundamental operation in image processing. Using this, we can reduce the amount of pixels while maintaining the structural aspect of the images.

The basic operation involved behind edge detection is called Convolution and is illustrated below :

![image.png](attachment:image.png)

Edges can be detected using various kinds of filters.

* First derivative based Sobel filter(for thicker edges)
* Second derivative based Laplacian filter(for finer edges)

Here, we want to consider the area containing only the leaf, while ignoring the background green. Hence, we use Sobel filter to identify the prominent edge of the leaf.

In [ ]:
def sobel_edge_detection(image):
  """
  Using Sobel filter

  Sobel filter takes the following arguments :
  1. Original Image
  2. Depth of the destination image
  3. Order of derivative x
  4. Order of derivative y
  5. Kernel size for convolutions

  f(Image, depth, order_dx, order_dy, kernel_size)
  """
  sobel_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize = 5)
  sobel_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize = 5)
  return sobel_x, sobel_y

In [ ]:
s_img_x, s_img_y = sobel_edge_detection(denoised_image)

plt.figure(figsize = (12, 8))
plt.subplot(2,2,1)
plt.imshow(sample_image, cmap = "gray")
plt.grid(False)
plt.title("Sample Image")

plt.subplot(2,2,2)
plt.imshow(denoised_image, cmap = "gray")
plt.grid(False)
plt.title("Denoised Image")

plt.subplot(2,2,3)
plt.imshow(s_img_x, cmap = "gray")
plt.grid(False)
plt.title("Sobel X filtered Image")

plt.subplot(2,2,4)
plt.imshow(s_img_y, cmap = "gray")
plt.grid(False)
plt.title("Sobel Y filtered Image")

# Automatically adjust subplot parameters to give specified padding.
plt.tight_layout()

Using sobel filter we found the edges, however for further pre-processing we aim to consider only the area of the leaf, that is the fine textured area we see in the gradient images. For that, we will use a much powerful inbuilt function of open-CV called Canny(). This function will return the edge coordinates.

Entire read is available on the OpenCV webpage :

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html#canny

# Canny Edge Detector :

The Canny filter is a multi-stage edge detector. It uses a filter based on the derivative of a Gaussian in order to compute the intensity of the gradients.The Gaussian reduces the effect of noise present in the image. Then, potential edges are thinned down to 1-pixel curves by removing non-maximum pixels of the gradient magnitude. Finally, edge pixels are kept or removed using hysteresis thresholding on the gradient magnitude.

The Canny has three adjustable parameters: the width of the Gaussian (the noisier the image, the greater the width), and the low and high threshold for the hysteresis thresholding.

The Canny edge detection algorithm is composed of 5 steps:

* Noise reduction;
* Gradient calculation;
* Non-maximum suppression;
* Double threshold;
* Edge Tracking by Hysteresis.

![image.png](attachment:image.png)

### Noise Reduction :

One way to get rid of the noise on the image, is by applying Gaussian blur to smooth it. To do so, image convolution technique is applied with a Gaussian Kernel (3x3, 5x5, 7x7 etc…). The kernel size depends on the expected blurring effect. Basically, the smallest the kernel, the less visible is the blur.

### Gradient Calculation :

The Gradient calculation step detects the edge intensity and direction by calculating the gradient of the image using edge detection operators.

The result is almost the expected one, but we can see that some of the edges are thick and others are thin. Non-Max Suppression step will help us mitigate the thick ones.

### Non-Maximum Supression :

Ideally, the final image should have thin edges. Thus, we must perform non-maximum suppression to thin out the edges.


### Double Threshold :

The double threshold step aims at identifying 3 kinds of pixels: strong, weak, and non-relevant:

* Strong pixels are pixels that have an intensity so high that we are sure they contribute to the final edge.

* Weak pixels are pixels that have an intensity value that is not enough to be considered as strong ones, but yet not small enough to be considered as non-relevant for the edge detection.

* Other pixels are considered as non-relevant for the edge.

Therefore, the significance of having two values in double threshold :

* High threshold is used to identify the strong pixels (intensity higher than the high threshold)

* Low threshold is used to identify the non-relevant pixels (intensity lower than the low threshold)

* All pixels having intensity between both thresholds are flagged as weak and the Hysteresis mechanism (next step) will help us identify the ones that could be considered as strong and the ones that are considered as non-relevant.



### Hysteresis :

Based on the threshold results, the hysteresis consists of transforming weak pixels into strong ones, if and only if at least one of the pixels around the one being processed is a strong one.

We will be using OpenCV's implementation of Canny edge detection. This was the theory involved behind the entire process.

Further information can be found on OpenCV's documentation : https://docs.opencv.org/trunk/da/d22/tutorial_py_canny.html

In [ ]:
from collections import deque
def canny_edge_detection(image):
  edges = cv2.Canny(image, 170, 200)
  return edges

def primary_roi(original_image, edge_image):
  edge_coordinates = deque()
  for i in tqdm(range(edge_image.shape[0])):
    for j in range(edge_image.shape[1]):
      if edge_image[i][j] != 0 :
        edge_coordinates.append((i, j))

  min_row = edge_coordinates[np.argsort([coordinate[0] for coordinate in edge_coordinates])[0]][0]
  max_row = edge_coordinates[np.argsort([coordinate[0] for coordinate in edge_coordinates])[-1]][0]
  min_col = edge_coordinates[np.argsort([coordinate[1] for coordinate in edge_coordinates])[0]][1]
  max_col = edge_coordinates[np.argsort([coordinate[1] for coordinate in edge_coordinates])[-1]][1]

  new_image = original_image.copy()
  new_edge_image = edge_image.copy()

  new_image[min_row - 10 : min_row + 10, min_col : max_col] = [255, 0, 0]
  new_image[max_row - 10 : max_row + 10, min_col : max_col] = [255, 0, 0]
  new_image[min_row : max_row , min_col - 10 : min_col + 10] = [255, 0, 0]
  new_image[min_row : max_row , max_col - 10 : max_col + 10] = [255, 0, 0]

  new_edge_image[min_row - 10 : min_row + 10, min_col : max_col] = [255]
  new_edge_image[max_row - 10 : max_row + 10, min_col : max_col] = [255]
  new_edge_image[min_row : max_row , min_col - 10 : min_col + 10] = [255]
  new_edge_image[min_row : max_row , max_col - 10 : max_col + 10] = [255]

  roi_image = new_image[min_row : max_row, min_col : max_col]
  edge_roi_image = new_edge_image[min_row : max_row, min_col : max_col]


  return roi_image, edge_roi_image

In [ ]:
plt.figure(figsize = (12, 8))
plt.subplot(1,2,1)
plt.imshow(sample_image, cmap = "gray")
plt.grid(False)
plt.title("Sample Image")

edge_image = canny_edge_detection(sample_image)

plt.subplot(1,2,2)
plt.imshow(edge_image, cmap = "gray")
plt.grid(False)
plt.title("Canny Edge Image")
# Automatically adjust subplot parameters to give specified padding.
plt.tight_layout()

In [ ]:
roi_image, edge_roi_image = primary_roi(sample_image, edge_image)

plt.figure(figsize = (12, 8))
plt.subplot(1,2,1)
plt.imshow(roi_image, cmap = "gray")
plt.grid(False)
plt.title("ROI Image")

plt.subplot(1,2,2)
plt.imshow(edge_roi_image, cmap = "gray")
plt.grid(False)
plt.title("Edge ROI Image")
# Automatically adjust subplot parameters to give specified padding.
plt.tight_layout()

# Histogram Equalization in ROI section of the whole image
### First of all, why can we not apply histogram equalization directly to an RGB image?
Histogram equalization is a non-linear process. Channel splitting and equalizing each channel separately is incorrect. Equalization involves intensity values of the image, not the color components. So for a simple RGB color image, histogram equalization cannot be applied directly on the channels. It needs to be applied in such a way that the intensity values are equalized without disturbing the color balance of the image. So, the first step is to convert the color space of the image from RGB into one of the color spaces that separates intensity values from color components. Some of the possible options are HSV/HLS, YUV, YCbCr, etc. YCbCr is preferred as it is designed for digital images. Perform histogram equalization on the intensity plane Y. Now convert the resultant YCbCr image back to RGB.

(Excerpt taken from :

https://prateekvjoshi.com/2013/11/22/histogram-equalization-of-rgb-images/ )

In [ ]:
def histogram_equalization(roi_image):
  image_ycrcb = cv2.cvtColor(roi_image, cv2.COLOR_RGB2YCR_CB)
  y_channel = image_ycrcb[:, :, 0] # apply histogram equalization on this channel
  cr_channel = image_ycrcb[:, :, 1]
  cb_channel = image_ycrcb[:, :, 2]
  # local histogram equalization
  clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
  equalized = clahe.apply(y_channel)
  equalized_image = cv2.merge([equalized, cr_channel, cb_channel])
  equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_YCR_CB2RGB)
  return equalized_image

In [ ]:
equalized_roi_image = histogram_equalization(roi_image)

plt.figure(figsize = (12, 8))
plt.subplot(1,2,1)
plt.imshow(roi_image, cmap = "gray")
plt.grid(False)
plt.title("ROI Image")

plt.subplot(1,2,2)
plt.imshow(equalized_roi_image, cmap = "gray")
plt.grid(False)
plt.title("Histogram Equalized ROI Image")
# Automatically adjust subplot parameters to give specified padding.
plt.tight_layout()

# Image Segmentation(Half-Toned Images) : Otsu's Binarization

In global thresholding, we used an arbitrary chosen value as a threshold. In contrast, Otsu's method avoids having to choose a value and determines it automatically.

We will apply Otsu's binarization segmentation method on the histogram equalized image obtained in the previous stage.

In [ ]:
otsu_threshold, otsu_image = cv2.threshold(cv2.cvtColor(equalized_roi_image, cv2.COLOR_RGB2GRAY), 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

plt.figure(figsize = (12, 8))
plt.subplot(1,2,1)
plt.imshow(equalized_roi_image, cmap = "gray")
plt.grid(False)
plt.title("equalized_roi_image")

plt.subplot(1,2,2)
plt.imshow(otsu_image, cmap = "gray")
plt.grid(False)
plt.title("Otsu's Thresholded Image")
# Automatically adjust subplot parameters to give specified padding.

plt.tight_layout()

# Image Segmentation(Colored Images) : K-means Clustering

In the previous section we explored image segmentation using Otsu's Binarization. However, this is applied normally on half toned, that is binary(black and white) images. In this section, we will explore a Machine Learning technique called K-means clustering to segment the different areas of the image.

Once again, the operation will be performed on the histogram equalized image of plant leaf.

K-means is mostly useful for applications like image compression or object recognition, because for these types of applications, it is inefficient to process the whole image.

### K-Means Segmentation Approach Using OpenCV

* `samples` : It should be of np.float32 data type, and each feature should be put in a single column. Here we have 3 channels, so every channel features have to be in one column. So, total columns we have are 3, while we don't care about the number of rows, hence -1. So, shape : (-1, 3).

* `nclusters(K)` : Number of clusters required at end.

* `criteria` : It is the iteration termination criteria. When this criteria is satisfied, algorithm iteration stops. Actually, it should be a tuple of 3 parameters. They are ( type, max_iter, epsilon )

Type of termination criteria. It has 3 flags as below:

* `cv.TERM_CRITERIA_EPS` - stop the algorithm iteration if specified accuracy, epsilon, is reached.
* `cv.TERM_CRITERIA_MAX_ITER` - stop the algorithm after the specified number of iterations, max_iter.
* `cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER` - stop the iteration when any of the above condition is met.
* `max_iter` - An integer specifying maximum number of iterations. epsilon - Required accuracy

1. `attempts` : Flag to specify the number of times the algorithm is executed using different initial labellings. The algorithm returns the labels that yield the best compactness. This compactness is returned as output.

2. `flags` : This flag is used to specify how initial centers are taken. Normally two flags are used for this : cv.KMEANS_PP_CENTERS and cv.KMEANS_RANDOM_CENTERS.

*Output parameters :

`compactness` : It is the sum of squared distance from each point to their corresponding centers.

`labels` : This is the label array (same as 'code' in previous article) where each element marked '0', '1'.....

`centers` : This is array of centers of clusters.*

In [ ]:
def segmentation(image, k, attempts) :
    vectorized = np.float32(image.reshape((-1, 3)))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
    res , label , center = cv2.kmeans(vectorized, k, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    segmented_image = res.reshape((image.shape))
    return segmented_image

In [ ]:
plt.figure(figsize = (12, 8))
plt.subplot(2,2,1)
plt.imshow(equalized_roi_image, cmap = "gray")
plt.grid(False)
plt.title("Histogram Equalized Image")

segmented_image = segmentation(equalized_roi_image, 3, 10) # k = 3, attempt = 10
plt.subplot(2,2,2)
plt.imshow(segmented_image, cmap = "gray")
plt.grid(False)
plt.title("Segmented Image with k = 3")

segmented_image = segmentation(equalized_roi_image, 4, 10) # k = 4, attempt = 10
plt.subplot(2,2,3)
plt.imshow(segmented_image, cmap = "gray")
plt.grid(False)
plt.title("Segmented Image with k = 4")

segmented_image = segmentation(equalized_roi_image, 5, 10) # k = 5, attempt = 10
plt.subplot(2,2,4)
plt.imshow(segmented_image, cmap = "gray")
plt.grid(False)
plt.title("Segmented Image with k = 5")

It's evident that all values of k do an amazing job in clustering the image into different segments. K= 5 appears to be the best upon closer inspection.

# Preparing Dataset for Deep Learning

In [ ]:
IMAGE_SIZE = (224, 224, 3)

In [ ]:
train_image = []
for id in tqdm(train_data["image_id"]):
    image = load_image(id)
    image = resize(image, IMAGE_SIZE)

    edge_image = canny_edge_detection(image)
    roi_image, _ = primary_roi(image, edge_image)
    equalized_roi_image = histogram_equalization(roi_image)

    train_image.append(image)

<ipython-input-37-f9a9c1bebfed>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm(train_data["image_id"]):


  0%|          | 0/1821 [00:00<?, ?it/s]

/content/drive/MyDrive/input/images/Train_0.jpg


<ipython-input-28-dafad952fe71>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(edge_image.shape[0])):


  0%|          | 0/224 [00:00<?, ?it/s]

...

/content/drive/MyDrive/input/images/Train_639.jpg


  0%|          | 0/224 [00:00<?, ?it/s]

/content/drive/MyDrive/input/images/Train_640.jpg


  0%|          | 0/224 [00:00<?, ?it/s]

/content/drive/MyDrive/input/images/Train_641.jpg


In [ ]:
#x_train = np.ndarray(shape = (len(train_image), IMAGE_SIZE[0], IMAGE_SIZE[1], 3), dtype = np.float32)
x_train = np.array(train_image)
x_train = x_train/255.0

y = train_data.copy()
del y["image_id"]
y_train = np.array(y.values)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
sample = x_train[5]
sample.shape

In [ ]:
plt.imshow(sample)
print("Label = ", y_train[5])

# Handling Imbalanced Dataset :

In Exploratory Data Analysis, we plotted a pie chart depticting number of classes in the training data set. It was observed that the 'multiple diseased' class was in minority, hence creating an imbalance between the class distributions. In order to cater that, we will be using SMOTE algorithm to try and increase the minority class in our data set.

SMOTE stands for Synthetic Minority Oversampling Technique.

![image.png](attachment:image.png)

This algorithm aims to balance class distribution by randomly increasing minority class examples by replicating them. SMOTE synthesizes new minority instances between existing minority instances.

An amazing video explaining it can be found on YouTube : https://www.youtube.com/watch?v=FheTDyCwRdE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 115)
x_train, y_train = smote.fit_resample(x_train.reshape((-1, IMAGE_SIZE[0] * IMAGE_SIZE[1] * 3)), y_train)
x_train = x_train.reshape((-1, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
print(x_train.shape)
print(y_train.shape)

In [ ]:
def label_count(label) :
    count = 0
    for entry in y_train :
        if np.array_equal(label, entry):
            count += 1
    return count

In [ ]:
healthy = np.array([1,0,0,0])
multiple_diseases = np.array([0,1,0,0])
rust = np.array([0,0,1,0])
scab = np.array([0,0,0,1])

In [ ]:
count_healthy = label_count(healthy)
count_multiple_diseases = label_count(multiple_diseases)
count_rust = label_count(rust)
count_scab = label_count(scab)

labels = ["Healthy", "Multiple Diseased", "Rust", "Scab"]
counts = [count_healthy, count_multiple_diseases, count_rust, count_scab]
explode = (0.05, 0.05, 0.05, 0.05)
fig, ax = plt.subplots(figsize = (20, 12))
ax.pie(counts, explode = explode, labels = labels, shadow = True, startangle = 90)
ax.axis("equal") # equal aspect ratio ensures pie graph is drawn as circle

Now, it's a well balanced dataset :-)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
VALIDATION_FACTOR = 0.1

val_size = int(len(x_train) * VALIDATION_FACTOR)

train_x = x_train[: len(x_train) - val_size]
train_y = y_train[: len(y_train) - val_size] # len(x_train) = len(y_train)

val_x = x_train[len(x_train) - val_size : len(x_train)]
val_y = y_train[len(y_train) - val_size : len(y_train)]

print("Shape of training data = ", train_x.shape, train_y.shape)
print("Shape of validation data = ", val_x.shape, val_y.shape)

In [ ]:
sample_image = train_x[250]
plt.imshow(sample_image)
plt.grid(False)
print("Image label = ", train_y[250])

# Early Stopping :

Stop training when a monitored metric has stopped improving.

* monitor : Quantity to be monitored.
* patience : Number of epochs with no improvement after which training will be stopped.
* restore_best_weights : Whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.
* verbose: int. 0: quiet, 1: update messages

More info on Tensorflow 2.O documentations : https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

In [ ]:
monitor_es = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 25,
                                              restore_best_weights = False, verbose = 0)

# Configure GPU for Deep Learning

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
gpus = tf.config.list_physical_devices("GPU")
print(gpus)
if len(gpus) == 1 :
    strategy = tf.distribute.OneDeviceStrategy(device = "/gpu:0")
else:
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision" : True})
print("Mixed precision enabled")

# Transfer Learning with Fine Tuning :

Transfer learning allows us to train deep networks using significantly less data then we would need if we had to train from scratch. With transfer learning, we are in effect transferring the “knowledge” that a model has learned from a previous task, to our current one.

Transfer learning has been consistently proven to boost model accuracy and reduce require training time. Less data, less time, more accuracy.

Here, we will be using VGG-16 neural network architecture.

## VGG-16 Architecture :

The VGG-16 model is a 16-layer (convolution and fully connected) network built on the ImageNet database.

![image.png](attachment:image.png)

Here, in the fine tuning step we will freeze the initial pre-trained layers having the weights learned from the imagenet dataset, however the last few convolution-pooling layers will remain unfreezed, allowing us to train them on our dataset. Note that, we will be designing a custom head as the original VGG aimed to predict out of 1000 classes, whereas we only have four.

Also, it's been proven that adding a Global Average Pooling layer before the fully connected layer imroves the accuracy of the model considerably.

**Global Average Pooling. Global Average Pooling is an operation that calculates the average output of each feature map in the previous layer. This fairly simple operation reduces the data significantly and prepares the model for the final classification layer**

In [ ]:
vgg = tf.keras.applications.vgg16.VGG16(include_top = False, weights = "imagenet", input_shape = IMAGE_SIZE)

In [ ]:
vgg.summary()

In [ ]:
vgg.input

In [ ]:
vgg.layers[-1].output

In [ ]:
output = vgg.layers[-1].output
output = tf.keras.layers.GlobalAveragePooling2D()(output)
vgg_model = tf.keras.models.Model(vgg.input, output)

Unfreezing the last few layers to custom train according to our needs, that is over four classes.

In [ ]:
vgg_model.trainable = True
set_trainable = False
for layer in vgg_model.layers :
    if layer.name in ['block5_conv1', 'block4_conv1']:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
pd.set_option("max_colwidth", -1)
info = [(layer, layer.name, layer.trainable) for layer in vgg_model.layers]
analysis = pd.DataFrame(info, columns = ["Layer Type", "Layer Name", "Train Status"])
analysis

## Bottleneck features :

The last activation feature map in the VGG-16 model (output from block5_pool) gives us the bottleneck features, which can then be flattened and fed to a fully connected deep neural network classifier.

We flatten the bottleneck features in the vgg_model object to make them ready to be fed to our fully connected classifier.

A way to save time in model training is to use this model and extract out all the features from our training and validation datasets and then feed them as inputs to our classifier.

In [ ]:
def get_bottleneck_features(deep_learning_model, input_images) :
    #tf.keras.backend.clear_session()
    #with strategy.scope() : #with tf.device("/device:GPU:0"):
    with tf.device("/device:GPU:0"):
        bottleneck_features = deep_learning_model.predict(input_images, verbose = 1)
        return bottleneck_features

training_bottleneck_features = get_bottleneck_features(vgg_model, train_x)
validation_bottleneck_features = get_bottleneck_features(vgg_model, val_x)

print("Shape of training bottleneck features = ", training_bottleneck_features.shape)
print("Shape of validation bottleneck features = ", validation_bottleneck_features.shape)

In [ ]:
vgg_model.output_shape[1]

In [ ]:
input_shape = vgg_model.output_shape[1]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape = (input_shape, )))

model.add(tf.keras.layers.Dense(512, activation = "relu", input_dim = input_shape))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Dense(512, activation = "relu"))
model.add(tf.keras.layers.Dropout(0.4))

model.add(tf.keras.layers.Dense(4, activation = "softmax"))

model.compile(loss='categorical_crossentropy', optimizer = tf.optimizers.Adam(lr = 0.001), metrics=['accuracy'])
model.summary()

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100

In [ ]:
with tf.device("/device:GPU:0"): #with tf.device("/device:GPU:0"):
    history = model.fit(x = training_bottleneck_features, y = train_y,
                        validation_data = (validation_bottleneck_features, val_y),
                        batch_size = BATCH_SIZE, epochs= EPOCHS, verbose = 1)

In [ ]:
X = np.arange(0,EPOCHS,1)
plt.figure(1, figsize = (20, 12))
plt.subplot(1,2,1)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(X, history.history["loss"], label = "Training Loss")
plt.plot(X, history.history["val_loss"], label = "Validation Loss")
plt.grid(True)
plt.legend()

plt.subplot(1,2,2)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(X, history.history["accuracy"], label = "Training Accuracy")
plt.plot(X, history.history["val_accuracy"], label = "Validation Accuracy")
plt.grid(True)
plt.legend()

In [ ]:
del train_x
del val_x
del train_y
del val_y

In [ ]:
del training_bottleneck_features
del validation_bottleneck_features

In [ ]:
import gc
gc.collect()

In [ ]:
test_data

# Test Our Model :

Of course, the testing data ought to go through the same pre-processing stages as the training data before we feed it to our model for classification.

In [ ]:
test_image = []
for id in tqdm(test_data["image_id"][:5]):
    image = load_image(id)
    image = resize(image, IMAGE_SIZE)

    edge_image = canny_edge_detection(image)
    roi_image, _ = primary_roi(image, edge_image)
    equalized_roi_image = histogram_equalization(roi_image)

    test_image.append(image)

In [ ]:
test_x = np.array(test_image)
test_x = test_x/255.0

In [ ]:
test_bottleneck_features = get_bottleneck_features(vgg_model, test_x)

In [ ]:
y_pred = model.predict(test_bottleneck_features)
y_pred.shape

In [ ]:
def convert_label(label) :
    m = max(label)
    index = list(label).index(m)
    if index == 0 :
        return "Healthy"
    elif index == 1 :
        return "Multiple Diseased"
    elif index == 2 :
        return "Rust"
    else:
        return "Scab"

In [ ]:
plt.figure(figsize = (12, 8))
plt.subplot(1,2,1)
plt.imshow(test_x[0], cmap = "gray")
label = convert_label(y_pred[0])
plt.grid(False)
plt.title("First Test Image")
print("Label of first testing image", label)

plt.subplot(1,2,2)
plt.imshow(test_x[2], cmap = "gray")
label = convert_label(y_pred[2])
plt.grid(False)
plt.title("Second Test Image")
print("Label of first testing image", label)

plt.tight_layout()

Thank You!